# Iteration 17: random number streams (4)

## Plain English Summary



## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import pandas as pd
import math

import simpy

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_17.md"))

modify the CCUModel class

The treatment functions should use one of the random number streams from the Experiment class for its sampling.

Show the full Experiment and CCUModel classes including all patient generator and treatment functions.


## Date and time of input to LLM

> 1215 18th December 2023 

## Sources

* https://towardsdatascience.com/object-oriented-discrete-event-simulation-with-simpy-53ad82f5f6e2
* https://www.promodel.com/onlinehelp/promodel/80/C-03%20-%20Use%20of%20Random%20Streams.htm
* https://stackoverflow.com/questions/29012212/implementing-common-random-numbers-in-a-simulation

## Observations on the LLM Output

* The LLM has correctly continued the numbering of streams e.g. 

```python
treatment_time = self.experiment.streams[6].lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] /   stay_distribution[0])**2), np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
```

## LLM Output 1:

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24,
                 trace=False,
                 random_number_set=0):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.trace = trace
        self.patient_count = 0
        self.random_number_set = random_number_set
        self.streams = []

        self.setup_streams(random_number_set)

    def reset_kpi(self):
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.mean_waiting_time_unplanned = 0
        self.total_unplanned_admissions = 0
        self.bed_occupancy = 0
        self.patient_count = 0

    def setup_streams(self, random_number_set):
        self.streams = []
        rng = np.random.default_rng(random_number_set)
        seeds = rng.integers(0, np.iinfo(np.int64).max, size=12)
        for seed in seeds:
            self.streams.append(np.random.default_rng(seed))

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(self.experiment.streams[0].exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(self.experiment.streams[1].exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(self.experiment.streams[2].exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(self.experiment.streams[3].exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(self.experiment.streams[4].exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(self.experiment.streams[5].normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                if self.experiment.trace:
                    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.experiment.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        arrival_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            wait_time = self.env.now - arrival_time
            if self.experiment.trace:
                print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = self.experiment.streams[6].lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
                self.experiment.mean_waiting_time_unplanned += wait_time
                self.experiment.total_unplanned_admissions += 1
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = self.experiment.streams[7].exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            if self.experiment.trace:
                print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Critical care bed is available for next patient at time {self.env.now}")

    def warmup_complete(self):
        yield self.env.timeout(self.experiment.warm_up_period)
        self.patient_count = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            mean_waiting_time_unplanned = self.experiment.mean_waiting_time_unplanned / self.experiment.total_unplanned_admissions
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds
            performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.experiment.cancelled_elective_count], 'Bed Utilization': [bed_utilization], 'Mean Waiting Time Unplanned': [mean_waiting_time_unplanned], 'Bed Occupancy': [bed_occupancy], 'Patient Count': [self.patient_count]})
            return performance_measures


## `multiple_replications`, and `results_summary` from prior iteration

In [4]:
import pandas as pd

def multiple_replications(experiment, num_replications=5):
    all_results = []

    for i in range(num_replications):
        experiment.setup_streams(i)  # Call the setup_streams method and pass in the current replication number
        model = CCUModel(simpy.Environment(), experiment)
        experiment.reset_kpi()
        results = model.run()
        results.insert(0, 'Replication', i+1)
        all_results.append(results)

    return pd.concat(all_results, ignore_index=True)


In [5]:
def results_summary(results):
    # Drop the 'Replication' column
    results = results.drop(columns='Replication')

    # Calculate the mean and standard deviation of each column
    mean = results.mean()
    std = results.std()

    # Create a summary dataframe
    summary = pd.DataFrame({'Mean': mean, 'Standard Deviation': std})

    return summary

In [6]:
# test code (written by tester)
experiment = Experiment()
replications = multiple_replications(experiment, 5)
results_summary(replications)

,Mean,Standard Deviation
Cancelled Elective Operations,337.800000,38.745322
Bed Utilization,0.909409,0.020721
Mean Waiting Time Unplanned,62.453489,55.234108
Bed Occupancy,21.825821,0.497310
Patient Count,1650.400000,17.826946


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `multiple_replications` function. 

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`
* **Repeatable results from multiple reps**: Two separate runs of 5 multiple replications
    * Expected: result:  The results summary tables are identical. The difference of the summary tables is 0.
    * Actual results **(PASS)**:  The results summary tables are identical. The difference of the summary tables is 0.

In [7]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [8]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    results = ccu_model.run()
    return ccu_model, results

In [9]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    return run_test(experiment)

In [10]:
model, results = extreme_value_test_1()
results

,Cancelled Elective Operations,Bed Utilization,Mean Waiting Time Unplanned,Bed Occupancy,Patient Count
0,0,0.211865,0.0,5.084763,370


### Test 2: Extreme value test 2

PASS

In [11]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means, trace=True)
    return run_test(experiment)

In [12]:
try:
    model, results = extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective surgery patient 1 arrived at time 9.516004601402404
Elective surgery patient 1 admitted to critical care bed at time 9.516004601402404
Elective surgery patient 2 arrived at time 29.363727055750303
Elective surgery patient 2 admitted to critical care bed at time 29.363727055750303
Elective surgery patient 3 arrived at time 45.45755491287409
Elective surgery patient 3 admitted to critical care bed at time 45.45755491287409
Elective surgery patient 3 discharged from critical care bed at time 50.28171470178084
Critical care bed is available for next patient at time 55.28171470178084
Elective surgery patient 4 arrived at time 69.11089461999146
Elective surgery patient 4 admitted to critical care bed at time 69.11089461999146
Elective surgery patient 4 discharged from critical care bed at time 78.98670125435662
Elective surgery patient 5 arrived at time 83.58658994817358
Elective surgery patient 5 admitted to critical care bed at time 83.58658994817358
Critical care bed is available

### Test 3: Different run length

PASS

In [13]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model, results = run_test(experiment)
    return model.env.now

In [14]:
try:
    test_results_collection_period(10)
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

### Test 4: Extreme value test 3

PASS.

In [15]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
 
    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    model, results = run_test(experiment)
    print(f'{experiment.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')
    return model, results

In [16]:
model, results = extreme_value_test_3()
results

experiment.total_treatment_time=7457.194345913003
experiment.results_collection_period=8640


,Cancelled Elective Operations,Bed Utilization,Mean Waiting Time Unplanned,Bed Occupancy,Patient Count
0,479,0.863101,4460.299269,0.863101,1665


### Test 5: Extreme value test 4

PASS

In [17]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M, trace=True)
    return run_test(experiment)

In [18]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 1 arrived from Accident and Emergency at time 8.525595075274827
Patient 1 admitted to critical care bed at time 8.525595075274827
Elective surgery patient 2 arrived at time 9.516004601402404
Elective surgery patient 2 admitted to critical care bed at time 9.516004601402404
Patient 3 arrived from Accident and Emergency at time 21.603681805557287
Patient 3 admitted to critical care bed at time 21.603681805557287
Patient 4 arrived from Emergency surgery at time 28.998286910237447
Patient 4 admitted to critical care bed at time 28.998286910237447
Elective surgery patient 5 arrived at time 29.363727055750303
Elective surgery patient 5 admitted to critical care bed at time 29.363727055750303
Patient 6 arrived from other hospitals at time 43.8501706273573
Patient 6 admitted to critical care bed at time 43.8501706273573
Elective surgery patient 7 arrived at time 45.45755491287409
Elective surgery patient 7 admitted to critical care bed at time 45.45755491287409
Patient 8 arrived from t

### Test 6: Vary Warm-up parameter

PASS

In [19]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model, results = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [20]:
test_warmup_period(0)

(8640, 8640)

## Test 7: Repeatable results from multiple reps

PASS

In [21]:
def test_repeatable_reps(n_reps):
    experiment = Experiment()
    replications = multiple_replications(experiment, n_reps)
    rs1 =  results_summary(replications)

    experiment = Experiment()
    replications = multiple_replications(experiment, n_reps)
    rs2 =  results_summary(replications)

    return rs1, rs2

In [22]:
rs1, rs2 = test_repeatable_reps(5)

In [23]:
rs1 - rs2

,Mean,Standard Deviation
Cancelled Elective Operations,0.0,0.0
Bed Utilization,0.0,0.0
Mean Waiting Time Unplanned,0.0,0.0
Bed Occupancy,0.0,0.0
Patient Count,0.0,0.0
